In [ ]:
import torch
import json
from peft import LoraConfig,TaskType
from datasets import Dataset
import itertools
import numpy as np
from transformers import AutoTokenizer,AutoModelForCausalLM,Trainer,DataCollatorForSeq2Seq,TrainingArguments


In [ ]:
# 加载模型和分词器
model_path="D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.half, device_map="cuda")

print(tokenizer.eos_token) # 这是模型决定的结束的词元token和它的id
print(tokenizer.eos_token_id)

print(tokenizer.pad_token) # 这是模型决定的结束的词元token和它的id
print(tokenizer.pad_token_id)

In [ ]:
# 加载训练数据
data=[]
data_path = "huanhuan.json"
with open(data_path) as f:
    data = json.load(f)
data

In [ ]:
# 转换成Dataset
train_dataset = Dataset.from_list(data)
train_dataset

In [ ]:
# 打印几条看看
data_temp = itertools.islice(train_dataset,5)
for i in data_temp:
    print(i)

In [ ]:
# 分词编码及预处理
def process_func(example):

    # 定义列表
    input_ids, labels = [], []


    # 设置编码最长编码
    MAX_LENGTH = 512 


    # instruction文本、编码
    text_instruction = "\n".join(["<|system|>", "现在你要扮演皇帝身边的女人--甄嬛", "<|user|>", example["instruction"][0] + example["input"][0] + "<|assistant|>"]).strip()+ "\n"
    instruction = tokenizer.encode(text=text_instruction,add_special_tokens=True, truncation=True, max_length=MAX_LENGTH)


    # response文本、编码
    text_response = example["output"][0]
    response = tokenizer.encode(text=text_response, add_special_tokens=False, truncation=True,max_length=MAX_LENGTH)
    


    # input_ids编码 = 指令 + 回复 + 结尾
    input_ids = instruction + response + [tokenizer.eos_token_id]

    
    # labels编码 = 填充（长度等于指令） + 回复 + 结尾
    labels = [tokenizer.pad_token_id] * len(instruction) + response + [tokenizer.eos_token_id]



    # 计算需要填充的长度512 - 64 = 448
    pad_len = MAX_LENGTH - len(input_ids) # 计算需要填充的长度
    

    # input_ids、labels 分别加上填充id，一起组成512长度的固定编码
    input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
    labels = labels + [tokenizer.pad_token_id] * pad_len

    # 处理labels
    labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]
    
    # 编码返回到数据集
    example["input_ids"]=np.array([input_ids])
    example["labels"]=np.array([labels])
    return example


In [ ]:
# map处理数据集
data=train_dataset.map(process_func,batched=True,batch_size=1,drop_last_batch=True)
data["input_ids"]

In [ ]:
# 配置Lora微调参数
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)

In [ ]:
# 配置训练参数
# Data collator GLM源仓库从新封装了自己的data_collator,在这里进行沿用。
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
    padding=False
)

args = TrainingArguments(
    output_dir="./output/ChatGLM",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=10,
    num_train_epochs=3,
    gradient_checkpointing=True,
    save_steps=100,
    learning_rate=1e-4,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data,
    data_collator=data_collator
)
trainer.train()

In [ ]:
# 模型推理
model.eval()
model = model.cuda()
question = "<|system|>\n现在你要扮演皇帝身边的女人--甄嬛\n<|user|>\n {}\n{}".format("你是谁？", "").strip() + "<|assistant|>\n"
ipt = tokenizer(question, return_tensors="pt").to(model.device)
answer=tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)
print(answer)

In [ ]:
# 重新加载
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained("./model/chatglm3-6b", trust_remote_code=True, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("./model/chatglm3-6b", use_fast=False, trust_remote_code=True)

p_model = PeftModel.from_pretrained(model, model_id="./output/ChatGLM/checkpoint-1000/")  # 将训练所得的LoRa权重加载起来

ipt = tokenizer("<|system|>\n现在你要扮演皇帝身边的女人--甄嬛\n<|user|>\n {}\n{}".format("你是谁？", "").strip() + "<|assistant|>\n", return_tensors="pt").to(model.device)
tokenizer.decode(p_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)
